In [1]:
 # Cell 1: Setup and Load Tonometry Data for OSD-679

import pandas as pd
import os
from dotenv import load_dotenv
import numpy as np # Good to have for data cleaning

print("Libraries imported.")

# Load environment variables from .env file
print("Loading .env variables...")
load_dotenv(override=True)
kg_git_dir = os.environ.get('KG_GIT')

# Check if KG_GIT path was loaded correctly
if not kg_git_dir or not os.path.isdir(kg_git_dir):
   raise ValueError(f"KG_GIT path not found or not set correctly in .env: Check the path and ensure it ends with '/'")
else:
    print(f"Project directory: {kg_git_dir}")
print(".env loaded.")

# --- Define Input File Paths ---
# Path to the Sample Table (created by save_metadata.py)
sample_table_file = os.path.join(kg_git_dir, 'OSD-679_SampleTable.csv')

# --- !!! PATH FOR TONOMETRY CSV (Saved directly in spoke_genelab) !!! ---
tonometry_file_path = os.path.join(kg_git_dir, 'LSDS-81_tonometry_Fuller_TonometryIOP_TRANSFORMED.csv')
# Data dictionary path (for reference if needed)
data_dict_file = os.path.join(kg_git_dir, 'OSD-679_sup_Fuller_DataDictionary.xlsx')

print(f"\nSample Table Path: {sample_table_file}")
print(f"Tonometry Data Path: {tonometry_file_path}")
print(f"Data Dictionary Path: {data_dict_file}")

# --- Load Input Data ---
sample_df = None
tono_df = None

print("\nLoading Sample Table CSV...")
try:
    if os.path.exists(sample_table_file):
        sample_df = pd.read_csv(sample_table_file)
        print(f"Loaded Sample Table: {sample_df.shape[0]} rows, {sample_df.shape[1]} columns")
        # display(sample_df.head(3)) # Optional: display head
    else:
         print(f"ERROR: Sample table file not found at:\n{sample_table_file}")
except Exception as e:
    print(f"ERROR loading Sample Table CSV: {e}")

print("\nLoading Tonometry Data CSV...")
try:
    if os.path.exists(tonometry_file_path):
        tono_df = pd.read_csv(tonometry_file_path)
        print(f"Loaded Tonometry Data: {tono_df.shape[0]} rows, {tono_df.shape[1]} columns")
        print("\nFirst 5 rows of Tonometry data:")
        display(tono_df.head())
        print("\nColumn Names:")
        print(list(tono_df.columns))
    else:
        print(f"ERROR: Tonometry file not found at the path:")
        print(tonometry_file_path)
except Exception as e:
    print(f"ERROR loading or reading Tonometry CSV: {e}")

# --- End of Cell 1 ---

Libraries imported.
Loading .env variables...
Project directory: /Users/gastondana/Desktop/spoke_genelab/
.env loaded.

Sample Table Path: /Users/gastondana/Desktop/spoke_genelab/OSD-679_SampleTable.csv
Tonometry Data Path: /Users/gastondana/Desktop/spoke_genelab/LSDS-81_tonometry_Fuller_TonometryIOP_TRANSFORMED.csv
Data Dictionary Path: /Users/gastondana/Desktop/spoke_genelab/OSD-679_sup_Fuller_DataDictionary.xlsx

Loading Sample Table CSV...
Loaded Sample Table: 644 rows, 2 columns

Loading Tonometry Data CSV...
Loaded Tonometry Data: 616 rows, 14 columns

First 5 rows of Tonometry data:


,Sample Name,Day_0_Intraocular_Pressure_torr,Day_7_Intraocular_Pressure_torr,Day_14_Intraocular_Pressure_torr,Day_28_Intraocular_Pressure_torr,Day_45_Intraocular_Pressure_torr,Day_90_Intraocular_Pressure_torr,Day_97_Intraocular_Pressure_torr,Day_104_Intraocular_Pressure_torr,Day_118_Intraocular_Pressure_torr,Day_135_Intraocular_Pressure_torr,Day_180_Intraocular_Pressure_torr,Unnamed: 12,Unnamed: 13
0,MT51_OD,13.0,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MT56_OD,16.0,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MT59_OD,22.0,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MT52_OD,15.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MT60_OD,18.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



Column Names:
['Sample Name', 'Day_0_Intraocular_Pressure_torr', 'Day_7_Intraocular_Pressure_torr', 'Day_14_Intraocular_Pressure_torr', 'Day_28_Intraocular_Pressure_torr', 'Day_45_Intraocular_Pressure_torr', 'Day_90_Intraocular_Pressure_torr', 'Day_97_Intraocular_Pressure_torr', 'Day_104_Intraocular_Pressure_torr', 'Day_118_Intraocular_Pressure_torr', 'Day_135_Intraocular_Pressure_torr', 'Day_180_Intraocular_Pressure_torr', 'Unnamed: 12', 'Unnamed: 13']


In [2]:
# Cell 2: Clean up and Reshape Tonometry Data
print("Cleaning and reshaping Tonometry data...")
# Check if tono_df exists from Cell 1
if 'tono_df' in locals() and tono_df is not None:
    try:
        # Drop the 'Unnamed: X' columns if they exist
        cols_to_drop = [col for col in tono_df.columns if 'Unnamed:' in col]
        if cols_to_drop:
            print(f"Dropping columns: {cols_to_drop}")
            # Create a cleaned copy to avoid modifying original df directly
            tono_df_cleaned = tono_df.drop(columns=cols_to_drop).copy()
        else:
            tono_df_cleaned = tono_df.copy()

        # Identify ID variable(s) that should remain columns
        id_vars = ['Sample Name']
        # Identify value variables (all the 'Day_X...' columns)
        value_vars = [col for col in tono_df_cleaned.columns if col not in id_vars]

        # Use pandas.melt to unpivot the dataframe from wide to long
        tono_df_long = pd.melt(
            tono_df_cleaned,
            id_vars=id_vars,
            value_vars=value_vars,
            var_name='MeasurementColumnHeader', # New column for original header
            value_name='IOP_torr'              # New column for the pressure value
        )

        # Drop rows where the IOP measurement itself is missing (NaN)
        print(f"Original number of potential measurements: {len(tono_df_long)}")
        tono_df_long.dropna(subset=['IOP_torr'], inplace=True)
        tono_df_long.reset_index(drop=True, inplace=True) # Tidy up index
        print(f"Number of actual measurements after dropping NAs: {len(tono_df_long)}")

        print("\nReshaped Tonometry data (first 5 rows):")
        display(tono_df_long.head()) # Display the reshaped data

    except Exception as e:
        print(f"ERROR cleaning or reshaping Tonometry data: {e}")
        tono_df_long = None # Indicate failure
else:
    print("Skipping reshape because tono_df is not available.")
    tono_df_long = None

Cleaning and reshaping Tonometry data...
Dropping columns: ['Unnamed: 12', 'Unnamed: 13']
Original number of potential measurements: 6776
Number of actual measurements after dropping NAs: 1630

Reshaped Tonometry data (first 5 rows):


,Sample Name,MeasurementColumnHeader,IOP_torr
0,MT51_OD,Day_0_Intraocular_Pressure_torr,13.0
1,MT56_OD,Day_0_Intraocular_Pressure_torr,16.0
2,MT59_OD,Day_0_Intraocular_Pressure_torr,22.0
3,MT52_OD,Day_0_Intraocular_Pressure_torr,15.0
4,MT60_OD,Day_0_Intraocular_Pressure_torr,18.0


In [3]:
# Cell 3: Parse Tonometry Measurement Column Header
print("Parsing Tonometry MeasurementColumnHeader...")
# Check if the DataFrame exists from the previous step
if 'tono_df_long' in locals() and tono_df_long is not None:
    try:
        # Define a regular expression to extract parts from the header string
        # Example: "Day_0_Intraocular_Pressure_torr"
        # Pattern: Starts with Day_, captures digits (\d+), captures middle part (.*), ends with _torr
        pattern = r"^Day_(\d+)_(.*)_torr$"

        # Use .str.extract() to apply the pattern and get capture groups
        extracted_data = tono_df_long['MeasurementColumnHeader'].str.extract(pattern)

        # Create new columns from the extracted data
        # Group 1 (\d+) is the Day number - convert it to numeric
        tono_df_long['Day'] = pd.to_numeric(extracted_data[0])
        # Group 2 (.*) is the Measurement Type - replace underscores with spaces
        tono_df_long['MeasurementType'] = extracted_data[1].str.replace('_', ' ')
        # We know the unit from the pattern
        tono_df_long['Unit'] = 'torr'

        print("Successfully parsed header components.")
        print("DataFrame with new columns (first 5 rows):")
        # Display the relevant columns, including the new ones
        display(tono_df_long[['Sample Name', 'Day', 'MeasurementType', 'Unit', 'IOP_torr']].head())

        # Optional: Check if any header strings failed to match the pattern
        failed_parse_count = tono_df_long['Day'].isna().sum()
        if failed_parse_count > 0:
            print(f"\nWARNING: {failed_parse_count} rows might not have parsed correctly (header didn't match pattern).")
        else:
            print("\nTonometry header parsing check passed.")

    except Exception as e:
        print(f"ERROR parsing Tonometry header column: {e}")
else:
    print("Skipping Tonometry header parsing because tono_df_long is not available.")

Parsing Tonometry MeasurementColumnHeader...
Successfully parsed header components.
DataFrame with new columns (first 5 rows):


,Sample Name,Day,MeasurementType,Unit,IOP_torr
0,MT51_OD,0,Intraocular Pressure,torr,13.0
1,MT56_OD,0,Intraocular Pressure,torr,16.0
2,MT59_OD,0,Intraocular Pressure,torr,22.0
3,MT52_OD,0,Intraocular Pressure,torr,15.0
4,MT60_OD,0,Intraocular Pressure,torr,18.0



Tonometry header parsing check passed.


In [4]:
# Cell 4: Merge Sample Group Information for Tonometry Data
print("Merging sample treatment group information...")
# Check if both needed dataframes exist from previous cells
if 'tono_df_long' in locals() and tono_df_long is not None and \
   'sample_df' in locals() and sample_df is not None:
    try:
        # Just in case, select only the necessary columns from sample_df
        # and drop duplicates based on Sample Name to ensure one group per sample
        sample_groups = sample_df[['Sample Name', 'Treatment Group']].copy().drop_duplicates(subset=['Sample Name'])

        # Perform a 'left' merge: keeps all rows from tono_df_long (our measurements)
        # and adds the 'Treatment Group' where 'Sample Name' matches in both tables.
        tono_data_with_groups = pd.merge(
            tono_df_long,
            sample_groups,
            on='Sample Name', # The common column to join on
            how='left'
        )

        print(f"Merge complete. DataFrame shape is now: {tono_data_with_groups.shape}")
        print("Example rows with Treatment Group added:")
        # Display relevant columns including the new Treatment Group
        display(tono_data_with_groups[['Sample Name', 'Day', 'MeasurementType', 'IOP_torr', 'Treatment Group']].head())

        # Optional: Check if any measurement rows failed to find a matching sample group
        failed_merge_count = tono_data_with_groups['Treatment Group'].isna().sum()
        if failed_merge_count > 0:
            print(f"\nWARNING: {failed_merge_count} measurement rows could not be matched with a treatment group.")
        else:
            print("\nMerge check passed (all tonometry rows matched a sample group).")

    except Exception as e:
        print(f"ERROR merging dataframes: {e}")
        tono_data_with_groups = None # Indicate failure
else:
    print("Skipping merge because prerequisite DataFrames (tono_df_long or sample_df) are not available.")
    tono_data_with_groups = None

Merging sample treatment group information...
Merge complete. DataFrame shape is now: (1630, 7)
Example rows with Treatment Group added:


,Sample Name,Day,MeasurementType,IOP_torr,Treatment Group
0,MT51_OD,0,Intraocular Pressure,13.0,3 month & Ambient Air & Not Applicable & Hindl...
1,MT56_OD,0,Intraocular Pressure,16.0,3 month & Ambient Air & Not Applicable & Hindl...
2,MT59_OD,0,Intraocular Pressure,22.0,3 month & Ambient Air & Not Applicable & Hindl...
3,MT52_OD,0,Intraocular Pressure,15.0,3 month & Ambient Air & Not Applicable & Norma...
4,MT60_OD,0,Intraocular Pressure,18.0,3 month & Ambient Air & Not Applicable & Norma...



Merge check passed (all tonometry rows matched a sample group).


In [5]:
# Cell 5: Map Eye (OD/OS) from Sample Name to Anatomy UBERON ID
print("Mapping Sample Name suffix (OD/OS) to Eye Anatomy UBERON ID...")
# Check if the DataFrame exists from the previous step
if 'tono_data_with_groups' in locals() and tono_data_with_groups is not None:
    try:
        # Extract the OD or OS suffix into a new 'Eye' column
        # This uses regex: extracts the OD or OS that occurs right before the end ($) of the string
        tono_data_with_groups['Eye'] = tono_data_with_groups['Sample Name'].str.extract(r'_(OD|OS)$')[0]

        # Define the mapping from the Eye suffix string to the UBERON ID
        eye_to_uberon = {
            'OD': 'UBERON:0004549', # Right Eye UBERON ID
            'OS': 'UBERON:0004548'  # Left Eye UBERON ID
        }

        # Create the 'AnatomyID' column by applying this mapping to the new 'Eye' column
        tono_data_with_groups['AnatomyID'] = tono_data_with_groups['Eye'].map(eye_to_uberon)

        print("Eye Anatomy IDs mapped.")
        print("DataFrame with Eye and AnatomyID columns added (first 5 rows):")
        # Display key columns to verify
        display(tono_data_with_groups[['Sample Name', 'Eye', 'AnatomyID', 'Day', 'IOP_torr']].head())

        # Optional: Check if any Sample Names didn't have OD/OS or failed mapping
        failed_eye_extract = tono_data_with_groups['Eye'].isna().sum()
        failed_map_count = tono_data_with_groups['AnatomyID'].isna().sum()
        if failed_eye_extract > 0:
            print(f"\nWARNING: Could not extract OD/OS suffix from {failed_eye_extract} Sample Names. Check format.")
        elif failed_map_count > 0:
             print(f"\nWARNING: {failed_map_count} extracted Eye values ('{tono_data_with_groups['Eye'].unique()}') couldn't be mapped.")
        else:
            print("\nEye Anatomy ID mapping check passed.")

    except Exception as e:
        print(f"ERROR mapping eye anatomy: {e}")
else:
    print("Skipping eye anatomy mapping because tono_data_with_groups is not available.")

Mapping Sample Name suffix (OD/OS) to Eye Anatomy UBERON ID...
Eye Anatomy IDs mapped.
DataFrame with Eye and AnatomyID columns added (first 5 rows):


,Sample Name,Eye,AnatomyID,Day,IOP_torr
0,MT51_OD,OD,UBERON:0004549,0,13.0
1,MT56_OD,OD,UBERON:0004549,0,16.0
2,MT59_OD,OD,UBERON:0004549,0,22.0
3,MT52_OD,OD,UBERON:0004549,0,15.0
4,MT60_OD,OD,UBERON:0004549,0,18.0



Eye Anatomy ID mapping check passed.


In [6]:
# Cell 6: Generate Unique IDs for Tonometry Data
print("Generating unique IDs for Tonometry data...")

if 'tono_data_with_groups' in locals() and tono_data_with_groups is not None:
    try:
        # Create a unique ID for each IOP measurement node
        # Combining relevant columns to ensure uniqueness
        tono_data_with_groups['IOP_ID'] = 'IOP_' + tono_data_with_groups['Sample Name'] + '_Day' + tono_data_with_groups['Day'].astype(str)

        # Create a unique ID for each anatomical entity node (Eye)
        tono_data_with_groups['Anatomy_Node_ID'] = 'Anatomy_' + tono_data_with_groups['AnatomyID']

        # Create a unique ID for each treatment group node
        tono_data_with_groups['Treatment_Node_ID'] = 'Treatment_' + tono_data_with_groups['Treatment Group']

        print("Unique IDs generated for IOP measurements, Anatomy, and Treatment Group.")
        print("DataFrame with new ID columns (first 5 rows):")
        display(tono_data_with_groups[['IOP_ID', 'Anatomy_Node_ID', 'Treatment_Node_ID', 'Sample Name', 'Day', 'IOP_torr']].head())

    except Exception as e:
        print(f"ERROR generating unique IDs: {e}")
else:
    print("Skipping unique ID generation because tono_data_with_groups is not available.")

Generating unique IDs for Tonometry data...
Unique IDs generated for IOP measurements, Anatomy, and Treatment Group.
DataFrame with new ID columns (first 5 rows):


,IOP_ID,Anatomy_Node_ID,Treatment_Node_ID,Sample Name,Day,IOP_torr
0,IOP_MT51_OD_Day0,Anatomy_UBERON:0004549,Treatment_3 month & Ambient Air & Not Applicab...,MT51_OD,0,13.0
1,IOP_MT56_OD_Day0,Anatomy_UBERON:0004549,Treatment_3 month & Ambient Air & Not Applicab...,MT56_OD,0,16.0
2,IOP_MT59_OD_Day0,Anatomy_UBERON:0004549,Treatment_3 month & Ambient Air & Not Applicab...,MT59_OD,0,22.0
3,IOP_MT52_OD_Day0,Anatomy_UBERON:0004549,Treatment_3 month & Ambient Air & Not Applicab...,MT52_OD,0,15.0
4,IOP_MT60_OD_Day0,Anatomy_UBERON:0004549,Treatment_3 month & Ambient Air & Not Applicab...,MT60_OD,0,18.0


In [7]:
# Cell 7: Prepare Data for Graph Nodes and Relationships
print("Preparing data for graph nodes and relationships...")

if 'tono_data_with_groups' in locals() and tono_data_with_groups is not None:
    try:
        # --- Prepare IOP Measurement Nodes ---
        iop_nodes = tono_data_with_groups[['IOP_ID', 'IOP_torr', 'Day']].rename(columns={'IOP_ID': 'id', 'IOP_torr': 'value', 'Day': 'day'})
        iop_nodes['entity'] = 'IOP'
        iop_nodes['unit'] = 'torr'

        print("\nPrepared IOP measurement nodes (first 5):")
        display(iop_nodes.head())

        # --- Prepare Anatomy Nodes ---
        # Get unique Anatomy IDs and UBERON IDs
        anatomy_nodes = tono_data_with_groups[['Anatomy_Node_ID', 'AnatomyID', 'Eye']].drop_duplicates(subset=['Anatomy_Node_ID']).rename(columns={'Anatomy_Node_ID': 'id', 'AnatomyID': 'uberon_id', 'Eye': 'eye'})
        anatomy_nodes['entity'] = 'Anatomy'

        print("\nPrepared Anatomy nodes:")
        display(anatomy_nodes)

        # --- Prepare Treatment Group Nodes ---
        # Get unique Treatment Groups and their IDs
        treatment_nodes = tono_data_with_groups[['Treatment_Node_ID', 'Treatment Group']].drop_duplicates(subset=['Treatment_Node_ID']).rename(columns={'Treatment_Node_ID': 'id', 'Treatment Group': 'name'})
        treatment_nodes['entity'] = 'Treatment'

        print("\nPrepared Treatment Group nodes:")
        display(treatment_nodes)

        # --- Prepare Relationships: IOP measured in Anatomy ---
        iop_anatomy_relationships = tono_data_with_groups[['IOP_ID', 'Anatomy_Node_ID']].rename(columns={'IOP_ID': 'source', 'Anatomy_Node_ID': 'target'})
        iop_anatomy_relationships['relation'] = 'measured_in'

        print("\nPrepared IOP-Anatomy relationships (first 5):")
        display(iop_anatomy_relationships.head())

        # --- Prepare Relationships: IOP belongs to a Treatment Group ---
        iop_treatment_relationships = tono_data_with_groups[['IOP_ID', 'Treatment_Node_ID']].rename(columns={'IOP_ID': 'source', 'Treatment_Node_ID': 'target'})
        iop_treatment_relationships['relation'] = 'belongs_to'

        print("\nPrepared IOP-Treatment relationships (first 5):")
        display(iop_treatment_relationships.head())

        print("\nData preparation for graph nodes and relationships complete.")

    except Exception as e:
        print(f"ERROR preparing graph data: {e}")
else:
    print("Skipping graph data preparation because tono_data_with_groups is not available.")

Preparing data for graph nodes and relationships...

Prepared IOP measurement nodes (first 5):


,id,value,day,entity,unit
0,IOP_MT51_OD_Day0,13.0,0,IOP,torr
1,IOP_MT56_OD_Day0,16.0,0,IOP,torr
2,IOP_MT59_OD_Day0,22.0,0,IOP,torr
3,IOP_MT52_OD_Day0,15.0,0,IOP,torr
4,IOP_MT60_OD_Day0,18.0,0,IOP,torr



Prepared Anatomy nodes:


,id,uberon_id,eye,entity
0,Anatomy_UBERON:0004549,UBERON:0004549,OD,Anatomy
6,Anatomy_UBERON:0004548,UBERON:0004548,OS,Anatomy



Prepared Treatment Group nodes:


...


Prepared IOP-Anatomy relationships (first 5):


,source,target,relation
0,IOP_MT51_OD_Day0,Anatomy_UBERON:0004549,measured_in
1,IOP_MT56_OD_Day0,Anatomy_UBERON:0004549,measured_in
2,IOP_MT59_OD_Day0,Anatomy_UBERON:0004549,measured_in
3,IOP_MT52_OD_Day0,Anatomy_UBERON:0004549,measured_in
4,IOP_MT60_OD_Day0,Anatomy_UBERON:0004549,measured_in



Prepared IOP-Treatment relationships (first 5):


,source,target,relation
0,IOP_MT51_OD_Day0,Treatment_3 month & Ambient Air & Not Applicab...,belongs_to
1,IOP_MT56_OD_Day0,Treatment_3 month & Ambient Air & Not Applicab...,belongs_to
2,IOP_MT59_OD_Day0,Treatment_3 month & Ambient Air & Not Applicab...,belongs_to
3,IOP_MT52_OD_Day0,Treatment_3 month & Ambient Air & Not Applicab...,belongs_to
4,IOP_MT60_OD_Day0,Treatment_3 month & Ambient Air & Not Applicab...,belongs_to



Data preparation for graph nodes and relationships complete.


In [8]:
# Cell 8: Save DataFrames to CSV Files
print("Saving DataFrames to CSV files...")

if 'iop_nodes' in locals() and 'anatomy_nodes' in locals() and 'treatment_nodes' in locals() and 'iop_anatomy_relationships' in locals() and 'iop_treatment_relationships' in locals():
    try:
        # Define the base filename for all CSVs
        base_filename = 'tonometry_data'

        # Save node DataFrames
        iop_nodes.to_csv(f'{base_filename}_iop_nodes.csv', index=False)
        anatomy_nodes.to_csv(f'{base_filename}_anatomy_nodes.csv', index=False)
        treatment_nodes.to_csv(f'{base_filename}_treatment_nodes.csv', index=False)

        # Save relationship DataFrames
        iop_anatomy_relationships.to_csv(f'{base_filename}_iop_anatomy_relationships.csv', index=False)
        iop_treatment_relationships.to_csv(f'{base_filename}_iop_treatment_relationships.csv', index=False)

        print("DataFrames saved to CSV files successfully.")

    except Exception as e:
        print(f"ERROR saving DataFrames to CSV: {e}")
else:
    print("Skipping saving to CSV because one or more DataFrames are not available.")

Saving DataFrames to CSV files...
DataFrames saved to CSV files successfully.
